# December 16, 2018
## Databases
I've been investigating how to output the data into databases. I found that relational databases aren't really the best place to store these types of data. These data are most definitely time series data, and they should be treated as such. [Time Series Databases](https://en.wikipedia.org/wiki/Time_series_database) provide an excellent starting point to look at this. 

## Database Choice
I chose to use [Timescale](https://www.timescale.com/) as our time series database. It's based on [PostgreSQL](https://www.postgresql.org/docs/10/index.html), which provides us with some advantages over other databases. It allows us to store JSON documents (e.g. header/firmware information). We're able to set the time column to be something other than a date (e.g. an integer). It might not be as redundant as something like MapR, but it's a good start. 

## Setup Timescale DB
I followed the Ubuntu [install instructions](https://docs.timescale.com/v1.1/getting-started/installation/ubuntu/installation-apt-ubuntu). Logging into the cli with
```bash
sudo service postgresql restart
psql -U postgres -W
```
Then creating the database
```sql
# create the database
CREATE database bagel;

# load the database
\c bagel

# convert to a timescaledb
create extension if not exists timescaledb cascade;

# shutoff telemetry tracking
ALTER [SYSTEM | DATABASE | USER] { *db_name* | *role_specification* } SET timescaledb.telemetry_level=off

# Create the table
CREATE TABLE data (
  id        smallint          NOT NULL,
  energy    DOUBLE PRECISION  NOT NULL,
  time      bigint            NOT NULL,
  cfd_time  DOUBLE PRECISION  NOT NULL
);

# create the hypertable for data table using the time column. Chunk time interval is set to 1 day for unix epoch time in ms. 
# Will need to think more about how this works. 
SELECT create_hypertable('data', 'time', chunk_time_interval => 86400000);

# Inserting a data point
INSERT INTO DATA VALUES(14,304.08124800000000398,19606962062518.000777,39213924125036.353274);
```

### Creating Users
```sql
create user vincent with password 'password';
grant all privileges on database bagel to vincent;
```

## Uploading data from sqlite
I uploaded the data from the sqlite by outputting as a CSV, truncating the decimal from the time column, and importing into the data table. 

# December 17, 2018
## Discussion w/ George
I had a discussion w/ George about the databases and how to setup the tables. I have a couple of options here. Having just a single big table might work for a while. His recommendation was that we have several tables. One table has {timestamps, crate, module, channel}. The other channel has all the details. That allows us to do quick lookups for what we need from the main table and then cross reference them to the details table. 

## Installing postgresql for windows
I'm going to install the database on windows as well. That way I can connect to the linux box without issues. I'm thinking that we might be able to connect w/o the database installed by using python. Let's check. 

## Testing the database connection

In [2]:
import keyring
import psycopg2
import yaml

with open('../cfg.yaml') as f:
    cfg = yaml.safe_load(f)
    
conn = psycopg2.connect(user=cfg['username'], password=keyring.get_password(cfg['host'], cfg['username']),
                                      host=cfg['host'], port=cfg['port'], database=cfg['db'])
cursor = conn.cursor()
cursor.execute('select * from data where id=14 limit 20')
print(cursor.fetchall())

[(14, 7576.572214, 19693238423165, 39386476846330.4), (14, 417.343303, 19693238484790, 39386476969580.0), (14, 305.573827, 19693238637156, 39386477274312.0), (14, 80.61771, 19693238698727, 39386477397454.5), (14, 342.357624, 19693238755370, 39386477510740.1), (14, 326.773797, 19693238906728, 39386477813456.2), (14, 1738.296982, 19693238914296, 39386477828592.8), (14, 232.901547, 19693238998831, 39386477997662.1), (14, 122.707598, 19693239018862, 39386478037724.8), (14, 135.301811, 19693239025359, 39386478050718.4), (14, 229.683297, 19693239082540, 39386478165080.2), (14, 561.489045, 19693239112649, 39386478225298.6), (14, 584.930979, 19693239202662, 39386478405324.2), (14, 1840.136679, 19693239228775, 39386478457550.2), (14, 6171.735934, 19693239298645, 39386478597290.2), (14, 160.691131, 19693239301822, 39386478603644.3), (14, 1532.055122, 19693239349315, 39386478698630.7), (14, 218.15213, 19693239404802, 39386478809604.4), (14, 869.480467, 19693239502743, 39386479005486.1), (14, 95.5

**HA!! It works!** Now we just need to figure out how to make this all more efficient. Right now it takes a long time to get the data out for a single clover channel. I'm going to try copying them out to another table and executing the same query.

## Copy values from one table to another
```sql
CREATE TABLE id04(id smallint NOT NULL, energy DOUBLE PRECISION NOT NULL, time bigint, cfd_time DOUBLE PRECISION NOT NULL);
INSERT INTO Table2 SELECT * FROM Table1 WHERE [Conditions]
```
Whelp, that didn't fare much better than the previous attempt. 

# December 29, 2018
## Test tables
Our test trigger time will be 19623818309088. We can find the gated gammas with

In [6]:
cursor = connection.cursor()
cursor.execute('select * from data where time between 19623818309088-126 and 19623818309088-63;')
print(cursor.fetchall())

[(10, 3401.897553, 19623818308976, 39247636617952.7), (9, 1284.855562, 19623818308966, 39247636617932.0)]


We'll create a table to hold results:
```sql
create table gated (id int, ge_energy double precision, ge_time bigint);
```

# December 30, 2018
## Slow as dirt
Things still are not too performant. I’m looking at 1000 trigger times and it takes about 5 min to generate our list. According to [this](https://stackoverflow.com/questions/8134602/psycopg2-insert-multiple-rows-with-one-query) forum we could see an improvement by generating the query ourself and executing it.
## Generating our own query
That does seem to have speeded things up a little bit. Let's try giving it the whole dataset. Generating the full statement list for 10 threads takes about 20 seconds each. Executing the queries ended in failure. I will need to determine a different way to execute this query. Error message says: 
> psycopg2.OperationalError: no connection to the server



# December 31, 2018
## Simple Kafka Cosumer
We're going to start working on our simple Kafka Consumer/Producer while it's fresh on my mind. 

# January 1, 2019
## Setting up Kafka on Windows
Following these [instructions](https://kafka.apache.org/quickstart). Using [Java SE Runtime 1.8.192](https://www.oracle.com/technetwork/java/javase/downloads/jre8-downloads-2133155.html)

```
# Starts the Zookeeper server 
D:\Programs\kafka_2.11-2.1.0\bin\windows\zookeeper-server-start.bat D:\Programs\kafka_2.11-2.1.0\config\zookeeper.properties

# Starts the actual Kafka server with the provided properties file.
D:\Programs\kafka_2.11-2.1.0\bin\windows\kafka-server-start.bat D:\Programs\kafka_2.11-2.1.0\config\server.properties

# Created a topic named test with replication 1, and a single partition. 
D:\Programs\kafka_2.11-2.1.0\bin\windows\kafka-topics.bat --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic test

# Starts the console based producer
D:\Programs\kafka_2.11-2.1.0\bin\windows\kafka-console-producer.bat --broker-list localhost:9092 --topic test

# Starts the console based consumer
D:\Programs\kafka_2.11-2.1.0\bin\windows\kafka-console-consumer.bat --bootstrap-server localhost:9092 --topic test --from-beginning
```
**NOTE: We need to have more kafka brokers (e.g. running instances of kafka) than our desired replication factors.**

I created a topic called daq to start testing communication between the producer and consumer. 

# January 2, 2019
## SQL efficiency
We could boost our efficiency finding overlapping events by doing a join on the main data table. The image below describes this process at a high level. 
![ansi join](ansi-join.png)

# January 5, 2019
## Consumer Working
The consumer is now consuming. We needed to set the bootstrap server to `localhost` so that it knew where the topics were located. 

# January 7, 2019
## Working on a c++ producer
Got the simple producer example working and able to read from the topic. Expanding it out now to use PAASS-LC Skeleton. Running into some linking issues. 
```
CMakeFiles/skeleton.dir/SkeletonUnpacker.cpp.o: In function `SkeletonUnpacker::ProcessRawEvent()':
SkeletonUnpacker.cpp:(.text+0x2bf): undefined reference to `cppkafka::Buffer::Buffer()'
SkeletonUnpacker.cpp:(.text+0x2c8): undefined reference to `cppkafka::Buffer::Buffer()'
collect2: error: ld returned 1 exit status
Analysis/Utilities/Skeleton/source/CMakeFiles/skeleton.dir/build.make:120: recipe for target 'Analysis/Utilities/Skeleton/source/skeleton' failed
make[3]: *** [Analysis/Utilities/Skeleton/source/skeleton] Error 1
CMakeFiles/Makefile2:829: recipe for target 'Analysis/Utilities/Skeleton/source/CMakeFiles/skeleton.dir/all' failed
make[2]: *** [Analysis/Utilities/Skeleton/source/CMakeFiles/skeleton.dir/all] Error 2
CMakeFiles/Makefile2:841: recipe for target 'Analysis/Utilities/Skeleton/source/CMakeFiles/skeleton.dir/rule' failed
make[1]: *** [Analysis/Utilities/Skeleton/source/CMakeFiles/skeleton.dir/rule] Error 2
Makefile:320: recipe for target 'skeleton' failed
make: *** [skeleton] Error 2
```
DUH, it's b/c I'm not linking to `libcppkafka`. Now we're failing with exit code 127. Good old exit code 127. Also, screw you clion for not showing my real errors. The SO isn't found during runtime:
>/home/vincent/projects/paass-lc/repo/cmake-build-release/Analysis/Utilities/Skeleton/source/skeleton: error while loading shared libraries: libcppkafka.so.0.2: cannot open shared object file: No such file or directory

Will need to get that listed in the `LD_LIBRARY_PATH`. That's all working and we're seeing information coming through in Kafka now!
```json
{
  'time': 2019-01-07 17:29:58,177,
  'module_name': ConsumerWorker,
  'level': INFO,
  'pid': 5604,
  'thread': Thread-1,
  'loggerName': root,
  'message': Writing from PAASS-LC Skeleton - Processing buffer from module 1 of length 5348
}
```
We're going to need to figure out how to pass the whole data block (sans first two words) to Kafka. that's going to be the closest emulation of what we'll get from the module. If we figure out how to handle these in our consumer, then we should be in good shape. This will also give us an idea of how to handle it in the producer as well. Figured out how to send the binary data using a string, but having trouble getting it through intact on the other side. Will need to look at a custom decoder? 

# January 10, 2019
## Binary data in python
Starting to understand how python handles binary data. It's mostly strings. I can use the struct method to unpack the data into the expected 32bit words. I created a test file that only contains a single 4 word header. The expected output is <540717, 123456789, 26001, 2345>. We'll use the [struct](https://docs.python.org/3/library/struct.html#examples) class to help us unpack these into integers that we can deal with. 

In [5]:
import struct
with open("../kafka/testfile.bin", "rb") as f:
    while True:
        chunk = f.read(4)
        if chunk:
            print(struct.unpack('i', chunk)[0])
        else:
            break

540717
123456789
26001
2345


Dealing with in memory binary data uses the `io.bytesIO` class. Let's try applying that to the data from kafka. Didn't get us anywhere meaningful. I'm going to try just sending the 4word unittest header through Kafka and see what comes out the other side. That should be much simpler to deal with. 

Looks like we've managed to get the data out properly? I'm not sure how this corresponds to the words, but things seem to be the same coming through Kafka.

| C++        | Python     |
|------------|------------|
| 1400106115 | 1400106115 |
| 3          | 3          |
| 8126945    | 8126945    |
| 30540247   | 30540247   |
| 30736852   | 30736852   |
| 30147030   | 30147030   |
| 30409163   | 30409163   |
| 30867925   | 30867925   |
| 30540241   | 30540241   |
| 31523292   | 31523292   |
| 30671325   | 30671325   |
| 30736852   | 30736852   |
| 30474713   | 30474713   |
| 29426117   | 29426117   |
| 29950400   | 29950400   |
| 30933459   | 30933459   |
| 30933464   | 30933464   |
| 30474713   | 30474713   |
| 29950416   | 29950416   |
| 30605774   | 30605774   |
| 30409168   | 30409168   |
| 31588831   | 31588831   |
| 31195614   | 31195614   |
| 29557204   | 29557204   |
| 30147015   | 30147015   |
| 30605779   | 30605779   |
| 30999000   | 30999000   |

I'm outputting the buffer before converting to a string for sending w/ Kafka: 
```c++
for (unsigned int i = 0; i < bufLen; i++)
            std::cout << buf[i] << std::endl;
```
This is being decoded in the consumer with 
```python
b = io.BytesIO(msg.value())
with b as buffer:
    while True:
        chunk = buffer.read(4)
        if chunk:
            print(struct.unpack('i', chunk)[0])
        else:
            break
```

Interestingly, the buffer doesn't contain an integer number of words. I'm not entirely sure how this happens with this generated file. Might be indicative of problems unpacking data. 

Success! We sent over the encoded unittest vector containing just the 4 word header and got back the appropriate response: 
```
got message and sending to output file now
540717
123456789
26001
2345
```
I suspect an issue with the data file. 
## Comparing data formats
The binary data is more efficient than the JSON structure. This surprised nobody ever. The binary data weighs in at 16 bytes. JSON at 426 bytes. That's an increase of about 27x. This is certianly a more efficient packing structure than for storage, but terrible for quick access (e.g. analysis). JSON compress to 250 bytes.  Compression factor of 1.7. This gives us only an increase of 16x from the binary. We'll have to consider that the data could be replicated as well. 

# January 11, 2019
## Reviewing data file. 
I've created the following data file 
```
48 45 41 44 01 00 00 00 04 00 00 00 00 00 B8 41 50 49 58 49 45 20 4C 49 53 54 20 44 41 54 41 20 55 20 4F 46 20 54 45 4E 4E 45 53 53 45 45 20 20 54 68 75 20 4A 61 6E 20 31 32 20 31 31 3A 33 39 3A 33 32 20 32 30 31 37 54 68 75 20 4A 61 6E 20 31 32 20 31 31 3A 33 39 3A 35 35 20 32 30 31 37 10 00 00 00 50 49 58 49 45 20 64 61 74 61 20 66 69 6C 65 20 FF FF FF FF 44 41 54 41 04 00 00 00 00 00 00 00 2D 40 08 00 15 CD 5B 07 91 65 00 00 29 09 00 00 45 4F 46 20 FF FF FF FF
```

It's about as simple as I can get. A HEAD buffer, with a single DATA buffer, then the end of the file. Sadly, `utkscan` cannot read the file. It's decoding the buffer, and then completely missing the DATA buffer. How the code actually unpacks anything properly is a complete mystery to me. 

This is getting wild. Looks like it's iterating past the first word of the DATA buffer (the length), and really only passing along the VSN. I'm debating on how far down this rabbit hole I should go. In principle, I should be able to get all of this information directly from the event. We just iterate through the file, find "DATA" find "0xFFFFFFFF" and everything between that goes into a message buffer. That sounds like the most straight forward way to go about getting these files converted. If we have corrupted data we'll have to sort out how to handle that. 

## Python data reader
Now that we know we can handle the data in python, we'll just move on to creating a file reader for python. We'll read everything between `DATA` and `0xFFFFFFFF` into a Kafka message then get on with our lives. This method should work for LDFs and PLDs. 

# January 12, 2019
## Data reader continued
We're able to read data files pretty reliably now. 

---
### D:/data/svp/kafka-tests/kafka-data-test-0.pld
* Number of Dirs         :  0 
* Number of Head         :  1 
* Number of Data Blocks  :  1 
* Number of Buffer Pads  :  3 
* Number of End of Files :  1 
* Total Words in File    :  41 
* Time To Read (s)       :  0.0

### D:/data/svp/kafka-tests/bagel-single-spill.pld
* Number of Dirs         :  0 
* Number of Head         :  1 
* Number of Data Blocks  :  1 
* Number of Buffer Pads  :  3 
* Number of End of Files :  1 
* Total Words in File    :  223456 
* Time To Read (s)       :  0.24099326133728027

### D:/data/utk/pixieworkshop/pulser_003.ldf
* Number of Dirs         :  1 
* Number of Head         :  1 
* Number of Data Blocks  :  193 
* Number of Buffer Pads  :  31215 
* Number of End of Files :  2 
* Total Words in File    :  1614219 
* Time To Read (s)       :  1.7529993057250977

### D:/data/ithemba/bagel/runs/runBaGeL_337.pld
* Number of Dirs         :  0 
* Number of Head         :  1 
* Number of Data Blocks  :  1204 
* Number of Buffer Pads  :  1206 
* Number of End of Files :  1 
* Total Words in File    :  282727801 
* Time To Read (s)       :  303.4150004386902

### D:/data/anl/vandle2015/a135feb_12.ldf
* Number of Dirs         :  1 
* Number of Head         :  1 
* Number of Data Blocks  :  65498 
* Number of Buffer Pads  :  22960597 
* Number of End of Files :  16386 
* Total Words in File    :  536731583 
* Time To Read (s)       :  571.7240064144135

---
For some of the longer files it takes about 5-7 minutes to scan through them. Keep in mind that we're just scanning word by word (4 bits per read). We can make the LDFs more efficient by increasing the chunk size to 8192, which is the fixed size of an LDF buffer. From what I can tell the PLD files have three words before the actual data starts. I'm trying to determine what these words are. 

## Moving data 
I'm moving the data from WSL to native windows install of postgres. I'm getting tired of popping open the WSL just to start postgres. It now starts automatically. 

## Import sql file into postgres
```sql
postgres=# \i c:/data/data01.sql
```

## Running Join Query
I'm trying out this join to see if we can get some results out of it. Here's the query: 
```sql
SELECT 
  id00.time, 
  id00.energy, 
  id10.time, 
  id10.energy
FROM 
  id10 
JOIN id00 ON id10.time BETWEEN id00.time+63 and id00.time+126
ORDER BY 
  id00.time;
```
It's been running for 28 minutes with no results yet. Will let it continue to roll and see what we get. It never finished...

# January 13, 2019
## Looking at PLD files 
From what I can tell the first three words are supposed to be the number of words in the buffer and then the data. I'm tracking down where "data" gets set.  Really, I guess we don't have to worry about this. It seems that all the PLDs are consistent in their formatting. 3 junk words then real data starts up. Here are the words from a few files. It does look like the newer versions of the writer are writing something different than the old ones. There's a reason that this is listed as an "experimental" format.  

* timing_003.pld
  * DATA
    * 65610
    * 2
    * 0
* runPR270_482.pld
  * DATA
    * 65088
    * 65088
    * 0
* lowgain_001.pld
  * DATA
    * 65542
    * 65540
    * 0
* lowgain_003.pld
  * DATA
    * 65542
    * 65540
    * 0
* runBaGeL_337.pld
  * DATA
    * 223420
    * 65542
    * 0
  * DATA
    * 226332
    * 65542
    * 0
  * DATA
    * 230776
    * 58286
    * 0
    
**Conclusion: We can use the idea of skipping three words before getting to the data for all PLD files.** 

What about LDF files?
* pulser_003.ldf
  * DATA
    * 8192
    * 32764
    * 10
    * 1
* sipmSkutek_001.ldf
  * DATA
    * 8192
    * 32748
    * 10
    * 0
  * DATA
    * 8192
    * 32748
    * 10
    * 1

Looks like LDFs skip 4 words. The fourth word is pretty useful since it tells us which chunk number we're working with. This is useful since LDFs can potentially split spills between buffers. Though, we never seem to actually do that. There's always some padding between the data and the end of the buffer (e.g. more than one `0xFFFFFFFF`). 

## Starting to build out Pixie Data Masks
I'm creating a class that will provide us decoding ability on the Pixie16 List Mode Data. The mask class is up and running properly, still need to write the unittests for it. 

## Bagel data
I'm looking at the first DATA block in the bagel_337 file and see some irregularities in the data. We're seeing a jump from everything looking good to everything being foobar. 
```json
{
  "crate": 0,
  "slot": 2,
  "channel": 14,
  "header_length": 4,
  "event_length": 4,
  "finish_code": 0,
  "event_time_low": 857012604,
  "event_time_high": 4565,
  "cfd_fractional_time": 7992,
  "cfd_trigger_source_bit": 1,
  "cfd_forced_trigger_bit": 0,
  "energy": 74,
  "trace_length": 0,
  "trace_out_of_range": 0
},
{
  "crate": 15,
  "slot": 3,
  "channel": 6,
  "header_length": 12,
  "event_length": 0,
  "finish_code": 0,
  "event_time_low": 1,
  "event_time_high": 16436,
  "cfd_fractional_time": 8,
  "cfd_trigger_source_bit": 0,
  "cfd_forced_trigger_bit": 0,
  "energy": 65,
  "trace_length": 6659,
  "trace_out_of_range": 0
},
```
As you can see we jump from crate 0 to crate 15. This makes no sense at all. I'm surprised that PAASS doesn't complain about this. Here's the encoded data:
> 2C 40 08 00 | B7 B4 13 33 | D5 11 B7 39 | 9E 00 00 00 

> 2A 40 08 00 | 6C 04 14 33 | D5 11 29 0E | ED 00 00 00 

> 2C 40 08 00 | D6 AC 14 33 | D5 11 EB 03 | 5A 00 00 00

> 2E 40 08 00 | 7C F9 14 33 | D5 11 38 5F | 4A 00 00 00

> 36 CF 00 00 | 01 00 00 00 | 34 40 08 00 | 41 00 03 1A

> D5 11 E5 64 | 00 02 00 00 | 33 40 08 00 | 98 96 03 1A

> D5 11 B2 71 | 44 00 00 00 | 3B 40 08 00 | 9C 96 03 1A 

> D5 11 7D 76 | A5 00 00 00 | 37 40 08 00 | 54 C5 03 1A 

> D5 11 81 5B | B8 02 00 00 | 39 40 08 00 | E3 DD 05 1A

> ....

> 55 40 08 00 | 99 D3 14 33 | D5 11 24 37 | CD 01 00 00

The next data block starts with 
> 29 40 08 00 | 21 12 15 33 | D5 11 62 57 | 7B 00 00 00

Based on where the data block ends up it looks like the data may have been corrupted or otherwise 

# January 14, 2019
## Digging in a bit more
This looks like an event that had an external timestamp on it or something. 
> 2E 40 08 00 | 7C F9 14 33 | D5 11 38 5F | 4A 00 00 00 | 36 CF 00 00 | 01 00 00 00

Bwah! I know what this is. It's the split to the next module!! Instead of breaking the modules into clearly distinct sections, they're packed with the length of the buffer + module number THEN the data. Those extra two words were throwing off my scanning.

## Generating postgres table
Got everything hooked up properly now. We're ready to start feeding these decoded data into the postgres database. I created a table from a simple header:
```sql
CREATE TABLE public.run337
(
  channel smallint,
  slot smallint,
  crate smallint,
  header_length smallint,
  event_length smallint,
  finish_code boolean,
  event_time_low bigint,
  event_time_high integer,
  cfd_fractional_time integer,
  cfd_trigger_source_bit boolean,
  cfd_forced_trigger_bit boolean,
  energy integer,
  trace_length smallint,
  trace_out_of_range boolean
)
```
I tried to match the data types to the minimum that they would fit. Postgres doesn't have an unsigned integer type (LAME), so we'll have to store `event_time_low` as a bigint. 